In [1]:
#only needed when unzipping the data file for the first time

# import zipfile
# with zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/ml-25m.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/My Drive/Colab Notebooks/')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv("ml-25m/movies.csv") 
ratings = pd.read_csv("ml-25m/ratings.csv") 

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
#View movies data

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
#View ratings data

ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
#Filter to movies with at least 14500 ratings

data = ratings.groupby('movieId').filter(lambda x: len(x) >= 14500)

In [15]:
print(ratings.shape)
print(data.shape)

(25000095, 4)
(8843222, 4)


In [16]:
#Filter to users with at least 250 ratings

data = data.groupby('userId').filter(lambda x: len(x) >= 250)
print(data.shape)

(407450, 4)


In [17]:
#Find number of distinct movies and users after subsetting the dataset

print("Number of Distinct Movies: " + str(len(set(data['movieId']))))
print("Number of Distinct Users: " + str(len(set(data['userId']))))
print(max(data['userId']))
print(max(data['movieId']))

Number of Distinct Movies: 335
Number of Distinct Users: 1482
162516
134130


In [6]:
#Relabel user and movie IDs so they start at 0 and are still unique

userID_list = data['userId'].unique()
movieID_list = data['movieId'].unique()

new_userId = np.zeros((len(data),1))
new_movieId = np.zeros((len(data),1))
for i in range(len(data)):
  new_userId[i,] = np.where(userID_list == data['userId'].iloc[i])
  new_movieId[i,] = np.where(movieID_list == data['movieId'].iloc[i])

data['userId2'] = new_userId
data['movieId2'] = new_movieId

#Save this relabeled dataset for use in models
data.to_csv('new_movies.csv') 

In [29]:
#Create h5py files where I will store user similarity scores (since numpy arrays can't hold all the data)

#NOTE: This section will not run properly if the cache files have already been created in your directory.  
#If you have already created the cache files, you must delete them before running this cell.

import h5py

num_users = len(set(data['userId2']))
num_movies = len(set(data['movieId2']))

hdf5_store = h5py.File("./cache.hdf5", "a")
users = hdf5_store.create_dataset("users", (num_users,num_movies), compression="gzip")
hdf5_store2 = h5py.File("./cache.hdf52", "a")
users_similarity = hdf5_store2.create_dataset("users_similarity", (num_users,num_users), compression="gzip")

In [30]:
#Populate users h5py file (giant array of ratings for each user and movie)

for row in range(len(data)):
  userID = data.iloc[row]['userId2']
  movieID = data.iloc[row]['movieId2']
  rating_num = data.iloc[row]['rating']
  users[int(userID),int(movieID)] = rating_num

print(users)

<HDF5 dataset "users": shape (1482, 335), type "<f4">


In [32]:
#Save users h5py file as csv for future use

np.savetxt("users.csv", users, delimiter=",")

In [33]:
#Populate users_similarity h5py file (giant array of similarity scores between users)

for i in range(len(set(data['userId2']))):
  for j in range(len(set(data['userId2']))):
    denominator = np.linalg.norm(users[i,:])*np.linalg.norm(users[j,:])
    if (denominator != 0):
      users_similarity[i,j] = np.sum((users[i,:])*(users[j,:])) / denominator
    else:
      users_similarity[i,j] = 0

print(users_similarity)

<HDF5 dataset "users_similarity": shape (1482, 1482), type "<f4">


In [34]:
#Save users_similarity h5py file as csv for future use

np.savetxt("users_similarity.csv", users_similarity, delimiter=",")